In [1]:
import torch
import torch.nn as nn
import torch.distributed as dist


import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm

from datasets import load_dataset, load_from_disk
from transformers import (
    AutoConfig, AutoTokenizer, DataCollatorForLanguageModeling, 
    get_cosine_schedule_with_warmup
)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss
from transformers import Adafactor
import json
import os
import copy

/home/woogoonkyu/anaconda3/envs/lora_null/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from vllm import LLM, SamplingParams
import pandas as pd
import torch
import torch._dynamo

# torch._dynamo.config.suppress_errors = True

2025-05-23 21:04:47,253	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [4]:
from datasets import load_dataset

ds = load_dataset("squarelike/ko_medical_chat")

Generating train split: 100%|██████████| 3038/3038 [00:00<00:00, 31950.31 examples/s]


In [5]:
dataset_dict = ds

In [6]:
from datasets import Dataset

def extract_sft_pairs(dataset):
    new_data = {"input": [], "output": [], "all_dialogue": []}

    for example in dataset:
        conv = example["conversations"]  # 전체 메시지 리스트

        # 전체 대화를 텍스트로 합쳐놓기
        full_dialogue = ""
        for turn in conv:
            speaker = turn['from'].capitalize()
            message = turn['value']
            full_dialogue += f"{speaker}: {message}\n"

        for i in range(3, len(conv)):
            try:
                if (conv[i-3]['from'] == 'client' and
                    conv[i-2]['from'] == 'doctor' and
                    conv[i-1]['from'] == 'client' and
                    conv[i]['from'] == 'doctor'):

                    input_text = (
                        f"Client: {conv[i-3]['value']}\n"
                        f"Doctor: {conv[i-2]['value']}\n"
                        f"Client: {conv[i-1]['value']}"
                    )
                    output_text = f"Doctor: {conv[i]['value']}"

                    new_data["input"].append(input_text)
                    new_data["output"].append(output_text)
                    new_data["all_dialogue"].append(full_dialogue.strip())  # 전체 대화도 추가
            except (IndexError, KeyError):
                continue

    return Dataset.from_dict(new_data)


In [7]:
sft_dataset = extract_sft_pairs(dataset_dict['train'])

print(f"생성된 샘플 수: {len(sft_dataset)}")
if len(sft_dataset) > 0:
    print(sft_dataset[0])


생성된 샘플 수: 11856
{'input': 'Client: 안녕하세요, 의사님. 오늘 아침에 일어나서 의자에 앉아 있을 때 방 전체가 돌아가는 것 같은 느낌이 들었어요.\nDoctor: 그런 증상이 있으셨군요. 이동할 때만 돌아가는 건가요?\nClient: 네, 제가 움직이려고 하면 세상 전체가 돌아가는 것 같아요.', 'output': 'Doctor: 그렇군요. 그리고 오전에 화장실에 가실 때 불안정하게 걸으셨다고 하셨는데, 그때 어떤 증상이 있었나요?', 'all_dialogue': 'Client: 안녕하세요, 의사님. 오늘 아침에 일어나서 의자에 앉아 있을 때 방 전체가 돌아가는 것 같은 느낌이 들었어요.\nDoctor: 그런 증상이 있으셨군요. 이동할 때만 돌아가는 건가요?\nClient: 네, 제가 움직이려고 하면 세상 전체가 돌아가는 것 같아요.\nDoctor: 그렇군요. 그리고 오전에 화장실에 가실 때 불안정하게 걸으셨다고 하셨는데, 그때 어떤 증상이 있었나요?\nClient: 집중하기가 어려웠고, 메스꺼움을 느꼈어요. 구토를 시도했지만 나오지 않았어요.\nDoctor: 그러셨군요. 그리고 복통도 있으셨다고 하셨는데, 어떤 종류의 복통이었나요?\nClient: 그냥 일반적인 복부 불쾌감이었어요. 하지만 방금 화장실에 갔다 온 후에는 돌아가는 증상이 완화되었어요.\nDoctor: 감사합니다. 이 증상은 어제부터 있었나요?\nClient: 아니요, 오늘 아침 처음으로 느껴졌어요.\nDoctor: 이해했습니다. 가장 가능성이 높은 원인은 양성돌발성 위치성 현훈(BPPV)입니다. 이 질환은 귀에서 발생하는 문제로 인해 발생하며, 몇 일 안에 스스로 호전됩니다. 베타히스틴 알약을 복용하면 증상이 완화될 것입니다. 전정 재활이나 적응 운동을 하면 증상의 재발을 예방할 수 있습니다. 이런 증상을 평가해 줄 이비인후과 전문의의 진료도 도\nDoctor: 도움이 될 것입니다. 최선을 다하겠습니다. 어떻게 대처해야 할지 궁금하신가요?\nClient: 그럼 어떻게

In [8]:
from datasets import DatasetDict

# 90% train, 10% test로 나누기
split_dataset = sft_dataset.train_test_split(test_size=0.2, seed=42)

# 결과 확인
split_dataset.save_to_disk("Dialogue_data_all")


Saving the dataset (0/1 shards):   0%|          | 0/9484 [00:00<?, ? examples/s]

Saving the dataset (1/1 shards): 100%|██████████| 2372/2372 [00:00<00:00, 182897.44 examples/s]


In [ ]:
e

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# CSV 불러오기 # sean0042/KorMedMCQA 을 전처리
df = pd.read_csv('merge_dataset.csv', encoding='utf-8-sig')

# train/test 분할 (기본적으로 80/20 비율, 랜덤 시드 고정)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# 파일 저장
train_df.to_csv('QA_train_dataset.csv', index=False, encoding='utf-8-sig')
test_df.to_csv('QA_test_dataset.csv', index=False, encoding='utf-8-sig')

print("✅ 데이터셋 분할 및 저장 완료:")
print(" - QA_train_dataset.csv")
print(" - QA_test_dataset.csv")


✅ 데이터셋 분할 및 저장 완료:
 - QA_train_dataset.csv
 - QA_test_dataset.csv


In [11]:
df

,type,input,output,A,B,C,D,E
0,dentist,65세 여자가 혀의 통증으로 내원하였다. 혈액검사 결과 혈색소 수치가 10.5 g/...,Choice 3,항핵항체(ANA),류마티스인자(RF),비타민B12(cobalamin),갑상샘자극호르몬(TSH),알칼리인산분해효소(ALP)
1,dentist,55세 남자가 구취 때문에 내원하였다. 관능검사(organoleptic method...,Choice 5,cadaverine,propionic acid,dimethyl sulfide,hydrogen sulfide,methyl mercaptan
2,dentist,"SGOT(AST), SGPT(ALT), 혈청빌리루빈, 젖산탈수소효소(LDH)가 증가...",Choice 1,지혈이 잘 안될 수 있으므로 혈액 응고시간을 확인한다.,감염성 심내막염의 위험이 있으므로 예방적 항생제 요법을 실시한다.,기회감염이 나타날 수 있으므로 항진균제나 항바이러스제를 사용한다.,스트레스와 불안을 감소시키기 위해 치료 전에 스테로이드를 투여한다.,"오전 중에 약속하고, 완전히 누운 자세보다는 반 정도 누운 자세나 앉은 자세에서 치..."
3,dentist,치과치료 시 에피네프린의 사용을 피해야 하는 질환은?,Choice 5,만성기관지염,부신기능부전,쇼그렌증후군,전신홍반루푸스,갑상샘기능항진증
4,dentist,37세 남자가 자동차 운전 중 후방 추돌 사고로 인해 목이 뒤로 꺾이면서 발생한 양...,Choice 4,격투손상(fight injury),역과손상(runover injury),전도손상(overturn injury),편타손상(whiplash injury),대측손상(contrecoup injury)
...,...,...,...,...,...,...,...,...
7484,nurse,옥시토신 유도분만 중인 40주 초산부가 통증을 호소하여 사정한 결과는 다음과 같다....,Choice 4,관장 시행,양막절개술 실시,자궁근이완제 투여,옥시토신 투여 중단,프로스타글란딘 질정 투여
7485,nurse,"심부전 환자의 사정결과가 다음과 같을 때 우선적인 중재는?• 허약, 심한 피로, 야...",Choice 2,수분섭취 권장,산소포화도 측정,객담배양검사 실시,트렌델렌부르크자세 유지,복도 걷기와 같은 활동 격려
7486,nurse,호흡곤란으로 입원한 환자의 사정결과가 다음과 같을 때 우선적인 중재는?• 3개월 전...,Choice 1,산소 투여,배변완화제 투여,중등도 활동 격려,혈액배양검사 실시,비위관 영양 공급
7487,nurse,외래 진료를 받고 있는 환자의 사정결과가 다음과 같을 때 간호중재는? • 전신 소양...,Choice 5,침상안정을 한다.,철분제제를 투여한다.,치실을 사용하여 치아를 관리한다.,앉아있을 때 다리를 내리도록 한다.,수분을 하루 3 L 이상 섭취하도록 격려한다.


In [12]:
# 예: 90% train, 10% test
train_df, test_df = train_test_split(df[['input','output','A','B','C','D','E']], test_size=0.2, random_state=42)

train_dataset = Dataset.from_pandas(train_df.reset_index(drop=True))
test_dataset = Dataset.from_pandas(test_df.reset_index(drop=True))

dataset_dict = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})

dataset_dict.save_to_disk("QA_dataset")



Saving the dataset (1/1 shards): 100%|██████████| 1498/1498 [00:00<00:00, 183656.35 examples/s]
